In [1]:
import sys
print(sys.executable)
!conda info | grep 'active env'

/sc/arion/work/massen06/.conda/envs/qc_r/bin/python3.9
     active environment : qc_torch
    active env location : /sc/arion/work/massen06/.conda/envs/qc_torch


In [2]:
import os
import copy
import anndata as ad
import scanpy as sc
import pandas as pd
import pegasus as pg
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy.stats import median_abs_deviation
from datetime import datetime
import shutil
import logging
import h5py
import tables
import shutil
import gzip
import sys
sys.path.append('/sc/arion/work/massen06/CellBender/cellbender/remove_background')

from cli import run_remove_background

/sc/arion/work/massen06/.conda/envs/qc_r/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/sc/arion/work/massen06/.conda/envs/qc_r/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/sc/arion/work/massen06/.conda/envs/qc_r/lib/python3.9/site-

In [3]:
metadata_dir = "/sc/arion/projects/CommonMind/single-cell-neurogenomics/LIVING_BRAIN/metadata"
base_dir = "/sc/arion/projects/CommonMind/single-cell-neurogenomics/LIVING_BRAIN/pre_processed_files/STARsolo_bams"
local_dir = "/sc/arion/work/massen06/temp_dir"
save_dir = "/sc/arion/work/massen06/living_brain1/ambient_removal"
gene_info_fn = "/hpc/users/massen06/MM_GRCm39_full.txt" 
mouse_mitocarta_fn = "MouseMitoCarta3.0.csv" 

In [4]:
target_dirs = [fn for fn in os.listdir(base_dir) if "Solo.out" in fn]

In [5]:
target_dirs

['LB_912_0min_Solo.out',
 'LB_WTE_0min_Living_Solo.out',
 'LB_WTF_0min_Solo.out',
 'LB_WTD_6hr_Post_Solo.out',
 'LB_WTD_0min_Living_Solo.out',
 'LB_WTG_0min_Solo.out',
 'LB_WTF_1hr_Solo.out',
 'LB_WTC_0min_PM_Solo.out',
 'LB_913_5min_Solo.out',
 'LB_WTE_6hr_Post_Solo.out',
 'LB_912_5min_Solo.out',
 'LB_WTG_1hr_Solo.out',
 'LB_WTB_0min_PM_Solo.out',
 'LB_WTA_0min_PM_Solo.out',
 'LB_WTA_0min_Living_PFC_Solo.out',
 'LB_WTB_0min_Living_PFC_Solo.out',
 'LB_913_0min_Solo.out',
 'LB_WTC_0min_Living_Solo.out']

In [6]:
def copy_data_and_compress(src_path, target_path):
    
    shutil.rmtree(target_path)
    shutil.copytree(src_path, target_path)
    
    # Ensure the files are compressed in the gzip format
    # Needed for 10x loading
    for suffix in ["filtered", "raw"]:
        target =  os.path.join(target_path, suffix)
        fns = os.listdir(target)
        for fn in fns:
            if not ".gz" in fn:
                f0 = os.path.join(target, fn)
                f1 = f0 + ".gz"
                with open(f0, 'rb') as f_in, gzip.open(f1, 'wb') as f_out:
                    f_out.writelines(f_in)
        fns = os.listdir(target)
        print("Files in target path", fns)
    
    

In [9]:
for n, t in enumerate(target_dirs[10:]):
   
     # Copy the data into a local directory and ensure it's compressed
    src_path = os.path.join(base_dir, t, "GeneFull")
    copy_data_and_compress(src_path, local_dir)
    
    # determine how many cells are in the filtered set
    filtered_dir = target =  os.path.join(local_dir, "filtered")
    adata = sc.read_10x_mtx(filtered_dir)
    n_cells = adata.shape[0]
    print(f"Number of cells in filtered AnnData: {n_cells}")
    del adata
    
    idx = t.find("_Solo")
    save_fn = os.path.join(save_dir, t[3:idx])
    raw_dir = target =  os.path.join(local_dir, "raw")
    args = arguments(raw_dir, save_fn, n_cells, 15_000, epochs=150, use_cuda=True)
    run_remove_background(args)
    
    
   

Files in target path ['features.tsv.gz', 'barcodes.tsv.gz', 'matrix.mtx.gz']
Files in target path ['matrix.mtx', 'barcodes.tsv', 'features.tsv.gz', 'barcodes.tsv.gz', 'features.tsv', 'matrix.mtx.gz']


cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cellbender:remove-background: 2023-07-31 15:16:30
cellbender:remove-background: Running remove-background
cellbender:remove-background: Loading data from directory /sc/arion/work/massen06/temp_dir/raw
cellbender:remove-background: CellRanger v3 format


Number of cells in filtered AnnData: 4452


cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Including 30264 genes that have nonzero counts.
cellbender:remove-background: Prior on counts in empty droplets is 402
cellbender:remove-background: Prior on counts for cells is 4928
cellbender:remove-background: Excluding barcodes with counts below 201
cellbender:remove-background: Using 4452 probable cell barcodes, plus an additional 10548 barcodes, and 57780 empty droplets.
cellbender:remove-background: Largest surely-empty droplet has 516 UMI counts.
cellbender:remove-background: Running inference...
cellbender:remove-background: [epoch 001]  average training loss: 3439.5745
cellbender:remove-background: [epoch 002]  average training loss: 2836.6166  (9.0 seconds per epoch)
cellbender:remove-background: [epoch 003]  average training loss: 2622.5112
cellbender:remove-background: [epoch 004]  average training loss: 2568.8018
cellbender:remove-background: [epoch 005]  average training loss: 253

Files in target path ['features.tsv.gz', 'barcodes.tsv.gz', 'matrix.mtx.gz']
Files in target path ['matrix.mtx', 'barcodes.tsv', 'features.tsv.gz', 'barcodes.tsv.gz', 'features.tsv', 'matrix.mtx.gz']


...

cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-background: Including 24055 genes that have nonzero counts.
cellbender:remove-ba

Files in target path ['features.tsv.gz', 'barcodes.tsv.gz', 'matrix.mtx.gz']
Files in target path ['matrix.mtx', 'barcodes.tsv', 'features.tsv.gz', 'barcodes.tsv.gz', 'features.tsv', 'matrix.mtx.gz']


cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cellbender:remove-background: Command:
cellbender remove-background /hpc/users/massen06/.local/share/jupyter/runtime/kernel-c323c4af-3828-464a-9dab-53a8aa9234b9.json
cell

Number of cells in filtered AnnData: 4271


cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Including 30246 genes that have nonzero counts.
cellbender:remove-background: Including 30246 genes that have nonzero counts.
cellbender:remove-background: Including 30246 genes that have nonzero counts.
cellbender:remove-background: Including 30246 genes that have nonzero counts.
cellbender:remove-background: Including 30246 genes that have nonzero counts.
cellbender:remove-background: Including 30246 genes that have nonzero counts.
cellbender:remove-background: Includ

In [8]:
class arguments:
    
    def __init__(
        self,
        input_fn,
        output_fn,
        expected_cell_count,
        total_droplets,
        fpr = 0.01,
        epochs = 30,
        fraction_empties=0.5,
        model="full",
        blacklisted_genes=[],
        exclude_antibodies=False,
        low_count_threshold=15,
        training_fraction=0.9,
        learning_rate=1e-4,
        use_cuda = False,
    ):
        self.input_file = input_fn
        self.output_file = output_fn
        self.expected_cell_count = expected_cell_count
        self.total_droplets = total_droplets
        self.fpr = fpr,
        self.epochs = epochs
        self.fraction_empties = fraction_empties
        self.model = model
        self.blacklisted_genes = blacklisted_genes
        self.exclude_antibodies = exclude_antibodies
        self.low_count_threshold = low_count_threshold
        self.training_fraction = training_fraction
        self.learning_rate = learning_rate
        self.num_training_tries = 1
        self.z_dim = 100
        self.z_hidden_dims = [500]
        self.posterior_batch_size = 32
        self.cells_posterior_reg_calc = 100
        self.use_cuda = use_cuda
        self.use_jit = False
        self.final_elbo_fail_fraction = 0.1
        self.epoch_elbo_fail_fraction = 20
        